In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004271984100341797
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 43.58219404254967
200000 58.15440568100392
300000 58.42423411280728
400000 57.98085846129369
500000 57.701979160289845
600000 62.05177793485373
700000 57.15966287384016
800000 57.23440400979331
900000 57.247445060453856
1000000 56.97227716761402
1100000 41.055231121172646
1200000 34.598147275213556
1300000 55.02224784122292
1400000 54.80678579775853
1500000 56.02074117463381
1600000 56.59496282279526
1700000 60.33119863634804
1800000 55.6666141850822
1900000 56.81248094236432
2000000 56.302859626719666
2100000 56.72972937222695
2200000 42.970123775957155
2300000 28.733820281182346
2400000 51.061237451549395
2500000 52.95165873215394
2600000 50.75755407975482
2700000 51.81793110910132
2800000 54.41058481255837
2900000 51.60238305207585
3000000 50.01004019

29700000 11.803159468461958
29800000 36.55065260359948
29900000 45.302561646007355
30000000 54.645729488454414
30100000 54.67197134868644
30200000 54.877175212239464
30300000 55.834940862317836
30400000 56.41788330868375
30500000 58.06463619543723
30600000 24.096115701996176
30700000 58.268287140274026
30800000 57.34512645018075
30900000 55.64318211778272
31000000 43.4271424827872
31100000 37.59680449964265
31200000 57.22974472537037
31300000 56.885156643798624
31400000 52.99426505242993
31500000 55.19930352781844
31600000 51.64738419946423
31700000 54.922853709084485
31800000 50.678943156475114
31900000 20.4522842648679
32000000 56.06051566194737
32100000 48.79701325164426
32200000 55.71132989727966
32300000 51.03919458557063
32400000 54.75925338912436
32500000 56.09112588060128
32600000 52.66737483807297
32700000 52.08080957648384
32800000 33.23619076622384
32900000 55.16889780731928
33000000 53.63468969621638
33100000 50.249351740346576
33200000 46.56721494734036
33300000 31.7518793

59900000 44.72454697875291
60000000 50.637018542253465
60100000 48.27872009288372
60200000 49.8039146346352
60300000 45.55749419341144
60400000 45.535567539455286
60500000 24.911265001585033
60600000 48.01459362003108
60700000 47.623427292185895
60800000 45.49404119882028
60900000 50.73013152144126
61000000 48.150516681150606
61100000 25.937345090706014
61200000 50.90264693438283
61300000 50.762991147517624
61400000 55.58950241190685
61500000 50.477472340743624
61600000 53.401763009852644
61700000 51.90893630274527
61800000 51.36096167335983
61900000 52.24970374452827
62000000 34.91549377172688
62100000 35.97892047070752
62200000 48.5557569829177
62300000 45.870816097189554
62400000 46.912611476113035
62500000 45.43065449010227
62600000 41.596866383419965
62700000 25.781179017297294
62800000 45.64059199025838
62900000 47.197833558526646
63000000 45.652116229166815
63100000 47.635742407862715
63200000 48.00169399302969
63300000 21.46365892821846
63400000 52.902780395541896
63500000 49.1

90000000 53.24524084701852
90100000 54.771101385456454
90200000 49.49791974901013
90300000 49.67519901486417
90400000 48.405207551681094
90500000 49.56687445210433
90600000 46.73189181099058
90700000 43.813705870437666
90800000 25.2334849824009
90900000 41.030095518012054
91000000 46.66753820459856
91100000 44.7245446381406
91200000 25.507797784831492
91300000 38.99118023024684
91400000 45.809580954292024
91500000 44.081671659097225
91600000 30.971401781563593
91700000 36.683767179238345
91800000 44.10877582554956
91900000 48.04672318491214
92000000 48.15476964462922
92100000 49.054031604013126
92200000 49.56365117848778
92300000 48.891784651594136
92400000 50.070978840265134
92500000 49.62452244552907
92600000 50.253204144735115
92700000 49.98068476378963
92800000 46.28741401003459
92900000 45.95069347435429
93000000 29.11764966625536
93100000 34.88664135871974
93200000 45.048560289418894
93300000 44.43542750116846
93400000 24.380458245499977
93500000 40.21784316250815
93600000 45.804

119400000 44.47134777264774
119500000 42.05062260088255
119600000 18.406921298930286
119700000 44.44897326978335
119800000 13.286485619319416
119900000 43.32686892059576
120000000 24.832203413220824
120100000 42.93648888910007
120200000 44.521868285592085
120300000 42.2296827371507
120400000 46.46283617148354
120500000 43.62566643722162
120600000 44.59433879024109
120700000 46.67334303426659
120800000 46.220688657446495
120900000 45.13091707060984
121000000 42.21949087177872
121100000 44.86932257159793
121200000 39.98452712700884
121300000 39.43828768262182
121400000 41.43424893093955
121500000 37.54015478751812
121600000 42.62608874777867
121700000 39.16222173371845
121800000 21.82382391822277
121900000 42.681516516013936
122000000 13.451038636180856
122100000 35.941315794484716
122200000 28.755424630327983
122300000 39.05868238451726
122400000 41.890494764457046
122500000 39.21401062947864
122600000 42.593353012303126
122700000 39.422459116089286
122800000 41.81410479252357
122900000

...

638600000 1.8558278314464967
638700000 1.7350902095629883
638800000 1.3448927363823089
638900000 0.9351643709941506
639000000 1.3098615300186118
639100000 1.737911523930904
639200000 1.4617339599391241
639300000 1.8621920324930852
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 1.9110682271384245
639500000 1.9377189744786918
639600000 1.949164570699407
639700000 1.502959746675337
639800000 1.4461892610520541
639900000 1.177572264082904
640000000 1.4420077864748664
640100000 1.2919918638960362
640200000 1.2750120347865097
640300000 1.524924351490128
640400000 1.063601313356332
640500000 1.5195840582753593
640600000 1.9930120532915745
640700000 1.4061631825887322
640800000 0.9927886413018161
640900000 1.5289837488184068
641000000 1.3270432305427353
641100000 1.5586889666007422
641200000 1.3369304087484806
641300000 1.106391808606865
641400000 1.5612014661940123
641500000 1.565245678077513
641600000 1.9143603141114416
641700000 1.8917656734360933
641800000 

665400000 0.4312832088031305
665500000 0.4919482026734855
665600000 0.48730956310269236
665700000 0.6212736154211296
665800000 0.6183926670286026
665900000 0.6226372689140737
666000000 0.5456845330699537
666100000 0.4517460640961892
666200000 0.6130000981549482
666300000 0.6046158135649006
666400000 1.2615788354134936
666500000 2.040303711136093
666600000 1.601460272280995
666700000 1.2641884246634485
666800000 0.9526685214824439
666900000 1.077039428087571
667000000 1.2263726533791715
667100000 1.406872157628274
667200000 1.5326241346592684
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.6515014972448991
667400000 0.49647681226428547
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.7131828000816989
667600000 1.867915848710619
667700000 2.1978630420413077
667800000 1.9729839352425032
667900000 2.50862158989039
668000000 1.6538885633795044
668100000 0.4790519457846164
668200000 0.5850101515653836
668300000 0.58041158973

691200000 0.8839503593697277
691300000 1.8506574596289536
691400000 1.7244379801315386
691500000 1.743482543040641
691600000 1.5314724819802417
691700000 1.5718613624153772
691800000 1.0665214376039591
691900000 1.9179295997640868
692000000 2.269979397879922
692100000 1.8543022772998188
692200000 1.6331689661748734
692300000 0.5911311345017831
692400000 1.320663274290987
692500000 1.4674844375010085
692600000 1.6566824305525896
692700000 1.2224413609838525
692800000 1.1471305558341722
692900000 0.885506058506739
693000000 1.5440414944889316
693100000 1.7297279032631816
693200000 1.4756595013132712
693300000 1.2439354948757138
693400000 0.9804956197113017
693500000 1.2869876094632369
693600000 1.963173316794004
693700000 1.2197005550189501
693800000 1.5934307279099105
693900000 1.4206848255166418
694000000 1.1832595176429377
694100000 1.9182531155234834
694200000 1.3831268428585384
694300000 1.699175989436886
694400000 1.123658450031692
694500000 1.1479380188606811
694600000 1.540179475

718600000 0.2425445083993011
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.31972973758848305
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.1752047226897657
718900000 0.21072153140187064
719000000 0.1936156680101633
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.4041790817903757
719200000 0.40043814839279257
719300000 0.3526513838800967
719400000 0.3063297747601019
719500000 0.33670962670656446
719600000 0.3650269907258689
719700000 0.29644691195889844
719800000 0.34792233686926693
719900000 0.44820154842894083
720000000 0.5999892396025512
720100000 0.3314086128695879
720200000 0.18540866900885303
720300000 0.18654994031143124
720400000 0.20672200095719498
720500000 0.4002745490477145
720600000 0.5490679249409754
720700000 0.29205072451245123
72080000

738900000 0.33053475495807527
739000000 0.26155381640874675
739100000 0.2360046708967566
739200000 0.267319597615136
739300000 0.29728007528300354
739400000 0.33471448187802433
739500000 0.3037242951331046
739600000 0.2895428001963794
739700000 0.26174971614768505
739800000 0.24309703813326228
739900000 0.2744578135156982
740000000 0.29165957359117467
740100000 0.25711596625925964
740200000 0.21077265117479627
740300000 0.29373682905177273
740400000 0.23428025726805063
740500000 0.17017765402617518
740600000 0.3759580438212454
740700000 0.3053877878062937
740800000 0.27554298697480895
740900000 0.1893008746278445
741000000 0.21756204305040108
741100000 0.21060490414800181
741200000 0.20542422705414293
741300000 0.25689412073192264
741400000 0.295477232070899
741500000 0.18980651508879065
741600000 0.2614713659719249
741700000 0.3227123303065757
741800000 0.25991587006323535
best so far: 0
type: [2, 1, 1, 5, 13] 130
cases of this type: 3570125
741900000 0.18787775885406993
742000000 0.0